In [1]:
from sklearn.model_selection import train_test_split
import os
import cv2
import xmltodict
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier



In [2]:
image_path = 'path_to_new_image'

In [3]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)
model.eval()

c:\Users\FUJITSU\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\FUJITSU\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
# Функція для зміни розміру зображення
def resize_image(img, size=(400, 400)):
    return cv2.resize(img, size)

In [5]:
def process_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"Не вдалося завантажити зображення за шляхом: {img_path}")
    img = resize_image(img)
    img = img.astype('float32') / 255.0  # нормалізація
    img = np.transpose(img, (2, 0, 1))  # переміщення каналів
    img = torch.tensor(img)
    return img


In [6]:
def predict_image(image_path):
    image = process_image(image_path).unsqueeze(0)
    model.eval()
    with torch.no_grad():
        output = model(image)
        _, predicted_class_idx = torch.max(output, 1)
    return predicted_class_idx.item()

In [7]:
# Функція для обробки анотації
def process_annotation(annot_path, class_to_idx):
    with open(annot_path) as f:
        annotation = xmltodict.parse(f.read())
    # Витягування мітки класу з анотації
    objects = annotation['annotation']['object']
    if isinstance(objects, list):
        label = class_to_idx[objects[0]['name']]
    else:
        label = class_to_idx[objects['name']]
    return label

In [8]:
class ImageDataset(Dataset):
    def __init__(self, images, annotations, class_to_idx):
        self.images = images
        self.annotations = annotations
        self.class_to_idx = class_to_idx

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        annot_path = self.annotations[idx]
        image = process_image(img_path)
        label = process_annotation(annot_path, self.class_to_idx)
        return image, label

images_folder = 'images'  # Назва папки з зображеннями
annotations_folder = 'annotations'  # Назва папки з анотаціями

images = []
annotations = []

for filename in os.listdir(images_folder):
    if filename.endswith('.png'):
        image_path = os.path.join(images_folder, filename)
        annotation_path = os.path.join(annotations_folder, filename[:-4] + '.xml')
        if os.path.exists(annotation_path):
            images.append(image_path)
            annotations.append(annotation_path)

if len(images) != len(annotations):
    raise ValueError("Кількість зображень та анотацій не співпадає")

# Створення списку всіх унікальних класів
unique_classes = set()

for annot_path in annotations:
    with open(annot_path) as f:
        annotation = xmltodict.parse(f.read())
        objects = annotation['annotation']['object']
        if isinstance(objects, list):
            for obj in objects:
                unique_classes.add(obj['name'])
        else:
            unique_classes.add(objects['name'])

class_names = list(unique_classes)
class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}

train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size=0.2, random_state=42)

train_dataset = ImageDataset(train_images, train_annotations, class_to_idx)
test_dataset = ImageDataset(test_images, test_annotations, class_to_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
# Визначення моделі
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 100 * 100, 512)
        self.fc2 = nn.Linear(512, num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 100 * 100)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

num_classes = len(class_to_idx)  # Залежить від кількості класів дорожніх знаків
model = SimpleCNN(num_classes)

In [10]:
# Навчання моделі
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5  

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

Epoch 1, Loss: 5.452285102822564
Epoch 2, Loss: 0.6654881225390867
Epoch 3, Loss: 0.5139357515356757
Epoch 4, Loss: 0.3748157146302136
Epoch 5, Loss: 0.21805250204422258


In [11]:
# Оцінка моделі
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


Accuracy: 76.13636363636364%


In [16]:
test_image_path = 'path_to_new_image/path_to_new_image.png'
img = cv2.imread(test_image_path)
if img is None:
    raise FileNotFoundError(f"Не вдалося завантажити зображення за шляхом: {test_image_path}")
print("Зображення успішно завантажено з розмірами:", img.shape)

Зображення успішно завантажено з розмірами: (179, 281, 3)


In [17]:
def resize_image(img, size=(400, 400)):
    return cv2.resize(img, size)

resized_img = resize_image(img)
print("Зображення змінено за розміром до:", resized_img.shape)


Зображення змінено за розміром до: (400, 400, 3)


In [18]:

img = resized_img.astype('float32') / 255.0  # нормалізація
img = np.transpose(img, (2, 0, 1))  # переміщення каналів
img_tensor = torch.tensor(img)
print("Зображення перетворено на тензор з формою:", img_tensor.shape)


Зображення перетворено на тензор з формою: torch.Size([3, 400, 400])


In [19]:
# Використання моделі для передбачення
img_tensor = img_tensor.unsqueeze(0)  # додати розмір для batch
with torch.no_grad():
    outputs = model(img_tensor)
    _, predicted = torch.max(outputs, 1)
print(f'Передбачений клас: {predicted.item()}')


Передбачений клас: 0


In [10]:
import xml.etree.ElementTree as ET

In [16]:
def load_data(images_path, annotations_path, image_size=(64, 64)):
    images = []
    labels = []
    
    for annotation_file in os.listdir(annotations_path):
        if annotation_file.endswith('.xml'):
            annotation_path = os.path.join(annotations_path, annotation_file)
            image_file = annotation_file.replace('.xml', '.png')  # Змінено розширення на .png
            image_path = os.path.join(images_path, image_file)
            
            if os.path.exists(image_path):
                image = Image.open(image_path).convert('RGB')
                image = image.resize(image_size)
                image = np.array(image)
                images.append(image)
                
                tree = ET.parse(annotation_path)
                root = tree.getroot()
                
                label = root.find('object').find('name').text  # Отримуємо мітку з імені об'єкта
                labels.append(label)
            else:
                print(f"Зображення не знайдено: {image_path}")
        else:
            print(f"Анотація не є .xml файлом: {annotation_file}")
                    
    if not images:
        print("Не завантажено жодного зображення.")
    if not labels:
        print("Не завантажено жодної анотації.")
    
    X = np.array(images)
    y = np.array(labels)
    return X, y

# Вказівка шляхів до папок
images_path = 'images'
annotations_path = 'annotations'

# Завантаження даних
X, y = load_data(images_path, annotations_path)

# Розділення даних на тренувальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Перетворення зображень на одномірні вектори
X_train_flattened = X_train.reshape((X_train.shape[0], -1))
X_test_flattened = X_test.reshape((X_test.shape[0], -1))

# Модель Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_flattened, y_train)
rf_predictions = rf_model.predict(X_test_flattened)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f"Точність моделі Random Forest: {rf_accuracy:.4f}")


# Перетворення зображень на одномірні вектори
X_train_flattened = X_train.reshape((X_train.shape[0], -1))
X_test_flattened = X_test.reshape((X_test.shape[0], -1))

# Модель Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train_flattened, y_train)
gb_predictions = gb_model.predict(X_test_flattened)
gb_accuracy = accuracy_score(y_test, gb_predictions)
print(f"Точність моделі Gradient Boosting: {gb_accuracy:.4f}")


Точність моделі Random Forest: 0.7102
Точність моделі Gradient Boosting: 0.6989
